In [ ]:
# Lab 4: 민원 처리 보고서 자동 생성 (40분)
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- Lab 1~3의 모든 함수 통합
- 민원 → 분류 → 검색 → 보고서 자동 생성 파이프라인 구축
- 구조화된 보고서 형식 작성
"""

# ============================================================
# 환경 설정
# ============================================================
import sys,os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber -q

import getpass
from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
from typing import List, Dict

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = "".strip()
client = OpenAI(api_key=api_key)


print("환경 설정 완료!\n")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:0

In [2]:
# ============================================================
# Lab 1-3 함수 통합
# ============================================================

print("\n[진행중] Lab 1-3 함수 재구축...")

# --- Lab 1: 민원 분류 ---
def classify_complaint(text: str) -> Dict[str, str]:
    """민원 분류 함수"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )

    return {
        "원문": text,
        "분류_결과": response.choices[0].message.content
    }

# --- Lab 2-3: PDF 처리 및 Vector DB ---
def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    """PDF 텍스트 추출"""
    pages_data = []
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))
        for i in range(total_pages):
            text = pdf.pages[i].extract_text()
            pages_data.append({
                "page_num": i + 1,
                "text": text,
                "char_count": len(text)
            })
    return pages_data

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    """텍스트 청킹"""
    chunks = []
    chunk_id = 0
    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            if chunk_text.strip():
                chunks.append({
                    "id": f"chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1
            start = end - overlap
    return chunks

def get_embedding(text: str) -> List[float]:
    """임베딩 생성"""
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# ChromaDB 초기화 및 데이터 로드
print("[진행중] Vector DB 구축 중...")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

try:
    chroma_client.delete_collection("fire_law")
    chroma_client.delete_collection("fire_prevention")
except:
    pass

# 소방기본법
print("  - 소방기본법 처리 중...")
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=10)
fire_law_chunks = chunk_text(fire_law_pages)
fire_law_collection = chroma_client.create_collection("fire_law")

for i, chunk in enumerate(fire_law_chunks):
    embedding = get_embedding(chunk['text'])
    fire_law_collection.add(
        ids=[f"law_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "소방기본법"}]
    )

# 화재예방
print("  - 화재예방 가이드 처리 중...")
prevention_pages = extract_text_from_pdf("화재예방.pdf")
prevention_chunks = chunk_text(prevention_pages)
prevention_collection = chroma_client.create_collection("fire_prevention")

for i, chunk in enumerate(prevention_chunks):
    embedding = get_embedding(chunk['text'])
    prevention_collection.add(
        ids=[f"prev_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "화재예방가이드"}]
    )

print("[완료] Vector DB 구축 완료!")

# --- Lab 3: 검색 함수 ---
def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    """소방기본법 검색"""
    query_embedding = get_embedding(query)
    results = fire_law_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })
    return search_results

def search_fire_prevention(query: str, n_results: int = 3) -> List[Dict]:
    """화재예방 가이드 검색"""
    query_embedding = get_embedding(query)
    results = prevention_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })
    return search_results

def search_both_sources(query: str, n_results_per_source: int = 2) -> Dict:
    """통합 검색"""
    law_results = search_fire_law(query, n_results_per_source)
    prevention_results = search_fire_prevention(query, n_results_per_source)
    return {
        "법령": law_results,
        "예방가이드": prevention_results,
        "total_count": len(law_results) + len(prevention_results)
    }

print("[완료] 모든 함수 준비 완료!\n")




[진행중] Lab 1-3 함수 재구축...
[진행중] Vector DB 구축 중...
  - 소방기본법 처리 중...
  - 화재예방 가이드 처리 중...
[완료] Vector DB 구축 완료!
[완료] 모든 함수 준비 완료!



In [3]:
# ============================================================
# 실습 1: 보고서 생성 파이프라인 설계
# ============================================================

print("="*60)
print("[실습 1] 보고서 생성 파이프라인 이해")
print("="*60)

print("""
민원 처리 보고서 자동 생성 프로세스:

[입력] 민원 내용
   ↓
[1단계] 민원 분류 (Lab 1)
   - 카테고리, 긴급도, 키워드 추출
   ↓
[2단계] 관련 자료 검색 (Lab 2-3)
   - 소방기본법에서 법령 검색
   - 화재예방 가이드에서 실무 가이드 검색
   ↓
[3단계] 보고서 생성 (Lab 4)
   - 검색 결과를 컨텍스트로 활용
   - LLM이 공식 보고서 형식으로 작성
   ↓
[출력] 구조화된 민원 처리 보고서
""")



[실습 1] 보고서 생성 파이프라인 이해

민원 처리 보고서 자동 생성 프로세스:

[입력] 민원 내용
   ↓
[1단계] 민원 분류 (Lab 1)
   - 카테고리, 긴급도, 키워드 추출
   ↓
[2단계] 관련 자료 검색 (Lab 2-3)
   - 소방기본법에서 법령 검색
   - 화재예방 가이드에서 실무 가이드 검색
   ↓
[3단계] 보고서 생성 (Lab 4)
   - 검색 결과를 컨텍스트로 활용
   - LLM이 공식 보고서 형식으로 작성
   ↓
[출력] 구조화된 민원 처리 보고서



In [4]:
# ============================================================
# 실습 2: 보고서 생성 함수 구현
# ============================================================

print("\n" + "="*60)
print("[실습 2] 보고서 생성 함수 구현")
print("="*60)

def generate_complaint_report(complaint_text: str) -> Dict:
    """
    민원 → 분류 → 검색 → 보고서 자동 생성

    Lab 1, 2, 3의 모든 함수를 활용한 통합 시스템
    """
    print("\n[진행중] 보고서 생성 파이프라인 실행...")

    # Step 1: 민원 분류 (Lab 1)
    print("  [1/3] 민원 분류 중...")
    classification = classify_complaint(complaint_text)
    print(f"       완료: {classification['분류_결과'].split(chr(10))[0]}")

    # Step 2: 관련 자료 검색 (Lab 2-3)
    print("  [2/3] 관련 법령 및 가이드 검색 중...")
    search_results = search_both_sources(complaint_text, n_results_per_source=2)
    print(f"       완료: 법령 {len(search_results['법령'])}개, 가이드 {len(search_results['예방가이드'])}개 검색")

    # Step 3: 보고서 본문 생성
    print("  [3/3] 보고서 작성 중...")

    # 컨텍스트 구성
    law_context = "\n".join([
        f"- {r['source']} {r['page']}p: {r['text'][:200]}"
        for r in search_results['법령']
    ])

    prevention_context = "\n".join([
        f"- {r['source']} {r['page']}p: {r['text'][:200]}"
        for r in search_results['예방가이드']
    ])

    # LLM으로 보고서 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방청 민원 처리 담당자입니다.
다음 형식으로 공식 보고서를 작성하세요:

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
【민원 처리 보고서】
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. 민원 내용
   [민원 요약 - 2-3줄]

2. 분류 결과
   [카테고리, 긴급도, 키워드]

3. 관련 법령 및 근거
   [소방기본법 관련 조항 및 내용]

4. 처리 방안
   [구체적인 대응 방법 및 절차]

5. 예방 가이드
   [화재예방 참고사항 및 권고사항]

6. 담당 부서 및 처리 기한
   [추천 담당 부서]
   [예상 처리 기한]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
작성일시: [현재 날짜]
작성자: 소방청 AI Agent
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"""
            },
            {
                "role": "user",
                "content": f"""다음 민원에 대한 처리 보고서를 작성하세요.

[민원 내용]
{complaint_text}

[분류 정보]
{classification['분류_결과']}

[참고 - 소방기본법]
{law_context}

[참고 - 화재예방 가이드]
{prevention_context}"""
            }
        ],
        temperature=0.4,
        max_tokens=1000
    )

    report = response.choices[0].message.content

    print("       완료: 보고서 생성 완료!")

    return {
        "민원원문": complaint_text,
        "분류": classification,
        "법령검색": search_results['법령'],
        "예방검색": search_results['예방가이드'],
        "보고서": report
    }




[실습 2] 보고서 생성 함수 구현


In [5]:
# ============================================================
# 실습 3: 보고서 생성 테스트
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] 보고서 생성 테스트")
print("="*60)

# 테스트 민원 1
test_complaint_1 = "상가 건물 복도에 소화기가 없고, 비상구도 짐으로 막혀있습니다. 안전 점검을 요청합니다."

print(f"\n[테스트 1]")
print(f"민원: {test_complaint_1}")
print("="*60)

result_1 = generate_complaint_report(test_complaint_1)

print("\n[생성된 보고서]")
print(result_1['보고서'])





[실습 3] 보고서 생성 테스트

[테스트 1]
민원: 상가 건물 복도에 소화기가 없고, 비상구도 짐으로 막혀있습니다. 안전 점검을 요청합니다.

[진행중] 보고서 생성 파이프라인 실행...
  [1/3] 민원 분류 중...
       완료: 카테고리: [소방시설]  
  [2/3] 관련 법령 및 가이드 검색 중...
       완료: 법령 2개, 가이드 2개 검색
  [3/3] 보고서 작성 중...
       완료: 보고서 생성 완료!

[생성된 보고서]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
【민원 처리 보고서】
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. 민원 내용  
   상가 건물 복도에 소화기가 없고, 비상구가 짐으로 막혀 있어 안전 점검을 요청하는 민원이 접수되었습니다. 

2. 분류 결과  
   카테고리: 소방시설  
   긴급도: 높음  
   키워드: 소화기, 비상구, 안전 점검  

3. 관련 법령 및 근거  
   - 소방기본법 제4조: 소방시설의 설치 및 유지 관리에 대한 규정이 있으며, 모든 소방대상물에는 적절한 소방시설이 설치되어야 함을 명시하고 있습니다.  
   - 소방기본법 제21조: 비상구 및 소화기 등의 소방시설은 항상 사용 가능해야 하며, 이를 방해하는 행위는 금지되어 있습니다.  

4. 처리 방안  
   - 해당 상가 건물에 대한 긴급 안전 점검을 실시합니다.  
   - 점검 시 소화기 설치 여부 및 비상구의 통행 가능 여부를 확인하고, 필요한 경우 소화기 설치 및 비상구 정리를 요구하는 행정명령을 발부합니다.  
   - 점검 결과에 따라 추가적인 안전 조치가 필요할 경우, 관련 법령에 따라 조치를 취합니다.  

5. 예방 가이드  
   - 상가 건물 내 소화기 및 비상구는 항상 접근 가능하도록 유지해야 하며, 정기적으로 점검하여 소화기 충전 상태를 확인해야 합니다.  
   - 비상구는 항상 개방 상태를 유지하고, 물건이나 장애물로 막히

In [6]:
# ============================================================
# 실습 4: 다양한 민원 유형 테스트
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] 다양한 민원 유형 테스트")
print("="*60)

test_complaints = [
    "음식점을 새로 열려고 하는데 소방시설을 어떻게 설치해야 하나요?",
    "아파트 지하 주차장에 소방차 전용구역에 차가 주차되어 있어 신고합니다.",
    "회사 사무실 소방시설 정기 점검은 몇 개월마다 해야 하나요?"
]

for i, complaint in enumerate(test_complaints, 2):
    print(f"\n\n{'='*60}")
    print(f"[테스트 {i}]")
    print(f"민원: {complaint}")
    print("="*60)

    result = generate_complaint_report(complaint)

    print("\n[생성된 보고서]")
    print(result['보고서'])
    print("\n" + "="*60)





[실습 4] 다양한 민원 유형 테스트


[테스트 2]
민원: 음식점을 새로 열려고 하는데 소방시설을 어떻게 설치해야 하나요?

[진행중] 보고서 생성 파이프라인 실행...
  [1/3] 민원 분류 중...
       완료: 카테고리: [소방시설]  
  [2/3] 관련 법령 및 가이드 검색 중...
       완료: 법령 2개, 가이드 2개 검색
  [3/3] 보고서 작성 중...
       완료: 보고서 생성 완료!

[생성된 보고서]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
【민원 처리 보고서】
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. 민원 내용
   음식점을 새로 열려고 하는 민원인이 소방시설 설치 방법에 대해 문의하였습니다. 음식점 운영에 필요한 소방시설의 종류와 설치 기준에 대한 정보가 필요합니다.

2. 분류 결과
   카테고리: 소방시설  
   긴급도: 보통  
   키워드: 소방시설, 음식점, 설치  

3. 관련 법령 및 근거
   - 소방기본법 제4조의3: 소방청장 또는 소방본부장은 소방시설 및 소방공사에 관한 기술적 지원을 제공할 수 있으며, 음식점과 같은 다중이용업소에 대한 소방시설 설치 기준을 정할 수 있습니다.
   - 「소방시설 설치 및 관리에 관한 법률」 제20조 및 제22조: 방염 및 소방시설의 자체점검에 관한 사항을 규정하고 있습니다.

4. 처리 방안
   1. 민원인에게 소방시설 설치에 대한 기본적인 안내를 제공합니다.
   2. 음식점의 규모, 용도 및 위치에 따라 필요한 소방시설의 종류(예: 스프링클러, 소화기, 비상조명 등)를 안내합니다.
   3. 소방시설 설치를 위한 전문 업체와의 상담을 권장하고, 필요한 경우 소방서에 문의하여 설치 기준 및 절차에 대한 구체적인 정보를 제공합니다.
   4. 음식점 개업 전 소방안전 점검을 받도록 안내합니다.

5. 예방 가이드
   - 음식점 내에서 사용할 수 있는 소화기 및 스프링클러 등의 소방시설을 

In [9]:
# ============================================================
# 실습 5: 보고서 품질 분석
# ============================================================

print("\n\n" + "="*60)
print("[실습 5] 생성된 보고서 품질 분석")
print("="*60)

print("""
보고서 품질 체크리스트:

[ ] 1. 민원 내용이 정확히 요약되었는가?
[ ] 2. 분류(카테고리/긴급도)가 적절한가?
[ ] 3. 관련 법령이 정확히 인용되었는가?
[ ] 4. 처리 방안이 구체적이고 실행 가능한가?
[ ] 5. 예방 가이드가 실무에 도움이 되는가?
[ ] 6. 담당 부서와 처리 기한이 합리적인가?
[ ] 7. 전체적인 문장이 공식적이고 명확한가?

개선 방법:
1. Temperature 조정 (더 일관된 답변: 0.2 / 더 창의적: 0.6)
2. 프롬프트 개선 (더 구체적인 지침 추가)
3. 검색 결과 개수 조정 (더 많은 컨텍스트: n_results=3)
4. 법령 우선순위 조정 (중요 조항 먼저 검색)
""")





[실습 5] 생성된 보고서 품질 분석

보고서 품질 체크리스트:

[ ] 1. 민원 내용이 정확히 요약되었는가?
[ ] 2. 분류(카테고리/긴급도)가 적절한가?
[ ] 3. 관련 법령이 정확히 인용되었는가?
[ ] 4. 처리 방안이 구체적이고 실행 가능한가?
[ ] 5. 예방 가이드가 실무에 도움이 되는가?
[ ] 6. 담당 부서와 처리 기한이 합리적인가?
[ ] 7. 전체적인 문장이 공식적이고 명확한가?

개선 방법:
1. Temperature 조정 (더 일관된 답변: 0.2 / 더 창의적: 0.6)
2. 프롬프트 개선 (더 구체적인 지침 추가)
3. 검색 결과 개수 조정 (더 많은 컨텍스트: n_results=3)
4. 법령 우선순위 조정 (중요 조항 먼저 검색)



In [10]:
# ============================================================
# Lab 4 완료
# ============================================================

print("\n" + "="*60)
print("[완료] Lab 4 완료!")
print("="*60)
print("""
학습 내용:
1. Lab 1-3 함수 통합
2. 민원 처리 파이프라인 구축
3. 구조화된 보고서 자동 생성
4. 다양한 민원 유형 테스트
5. 보고서 품질 분석

핵심 성과:
- 민원 입력 → 자동 분류 → 법령 검색 → 보고서 생성
- 완전 자동화된 민원 처리 시스템 구축
- 법령 + 실무 가이드 융합 보고서

핵심 함수:
- generate_complaint_report(): 통합 보고서 생성 (Lab 5에서 UI 연결)

다음 Lab:
- Lab 5에서는 이 시스템에 Gradio UI를 추가하여
  누구나 쉽게 사용할 수 있는 웹 인터페이스를 만듭니다.
""")


[완료] Lab 4 완료!

학습 내용:
1. Lab 1-3 함수 통합
2. 민원 처리 파이프라인 구축
3. 구조화된 보고서 자동 생성
4. 다양한 민원 유형 테스트
5. 보고서 품질 분석

핵심 성과:
- 민원 입력 → 자동 분류 → 법령 검색 → 보고서 생성
- 완전 자동화된 민원 처리 시스템 구축
- 법령 + 실무 가이드 융합 보고서

핵심 함수:
- generate_complaint_report(): 통합 보고서 생성 (Lab 5에서 UI 연결)

다음 Lab:
- Lab 5에서는 이 시스템에 Gradio UI를 추가하여
  누구나 쉽게 사용할 수 있는 웹 인터페이스를 만듭니다.

